<a href="https://colab.research.google.com/github/debanga/DeepLearningNotebooks/blob/master/Inter_epoch_checkpoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable
from tqdm.notebook import tqdm

data = pd.DataFrame()
data['plastic'] = pd.Series([100,50,30])
data['paper'] = pd.Series([0,50,0])
data['glass'] = pd.Series([0,50,90])
data['student'] = pd.Series([1,0,0])
data['worker'] = pd.Series([0,0,1])
data['elder'] = pd.Series([0,1,0])

data_x = np.array(data[["plastic","paper","glass"]], dtype=np.float32)
data_y = np.array(data[["student","worker","elder"]], dtype=np.float32)
x_train = torch.from_numpy(data_x)
y_train = torch.from_numpy(data_y)

In [4]:
model = torch.nn.Sequential(
    torch.nn.Linear(3,3, bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(3,3, bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(3,3, bias=True),
    torch.nn.ReLU(),
    torch.nn.Softmax(dim=1)
)

print(model)

Sequential(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=3, bias=True)
  (3): ReLU()
  (4): Linear(in_features=3, out_features=3, bias=True)
  (5): ReLU()
  (6): Softmax(dim=1)
)


In [8]:
num_epoch = 1000

loss_function = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
t = tqdm(range(num_epoch))

for epoch in t:
    input = Variable(x_train)
    target = Variable(y_train)

    # forward
    out = model(input)
    loss = loss_function(out, target)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # show
    t.set_description('Epoch[{}/{}], loss: {:.6f}'
          .format(epoch + 1, num_epoch, loss.data.item()))

# predicting
print(model(torch.tensor([[500, 500, 500]], dtype=torch.float32)))


tensor([[8.4729e-31, 8.4729e-31, 1.0000e+00]], grad_fn=<SoftmaxBackward>)
